In [1]:
# Modules to import

!pip install --user nipy
!pip install --user nilearn
!pip install --user seaborn
!pip install --user keras

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Main Imports

import nilearn

from nilearn.image import resample_to_img

import pylab as plt

import numpy as np
import nibabel as nb
import os
import glob
import random
import pandas as pd
import re

import seaborn as sns #added
sns.set(style="darkgrid") #added

from nilearn.image import mean_img #added
from nilearn.plotting import plot_anat #added

# Principal code parameters

In [6]:
# Local directory where the Data are mounted
rootDirectory = "/data/RMN/LUCA_PASQUINI"

# Local subdirectory where dataset is mounted
dataDir = "DATI_SEGMENTATI_SCALATI_media"

# Dataset dir
datasetDir = f"{rootDirectory}/{dataDir}"

#CSV fileroot 
fileName = f"{rootDirectory}/{dataDir}/Array_Labels_Def.csv"

SUBJECT_NAME_REPEATED=["BIANCHI","BOVE","PROIETTI"]

#for i in range(len(SUBJECT_NAME_REPEATED)):
#    f"{SUBJECT_NAME_REPEATED[i]}"
#    print(f"{SUBJECT_NAME_REPEATED[i]}")

PATIENT_REMOVED=['Pascal','Mitchell','Rufini','Farella','Array']

SEQUENCE_1= "rCBV_registered"
SEQUENCE_2= "rCBV"
#T1_registered , T1
#T2_registered , T2
#FLAIR_registered , FLAIR
#ADC_registered , ADC
#rCBV_registered , rCBV
#MPRAGEMDC
MaskPath = "SOLID"
#SOLID
#NECROSI
#T1ROI
#T2ROI


#reference dimensions
dim1=192
dim2=256
dim3=144


'''Model parameters'''

#percentage of training test
p=0.8

# Specify shape of convolution kernel
kernel_size = (3, 3)

# Specify number of output categories
n_classes = 2

# Specify number of filters per layer
filters = 16

#added block
nEpochs = 100  # Increase this value for better results (i.e., more training)

batch_size = 16   # Increasing this value might speed up fitting

activation='relu'

activation_Dense='softmax'

# optimizer
learning_rate = 1e-5
#adam = Adam(lr=learning_rate)
#sgd = SGD(lr=learning_rate)

loss='categorical_crossentropy'

metrics=['accuracy']


# Survival Labels dataframe

# Functions

In [7]:
'''' Get index positions of value in dataframe '''
def getIndexes(dfObj, value):
    listOfPos = list()
    result = dfObj.isin([value]) # Get bool dataframe with True at positions where the given value exists
    seriesObj = result.any() # Get list of columns that contains the value
    columnNames = list(seriesObj[seriesObj == True].index)
    for col in columnNames: # Iterate over list of columns and fetch the rows indexes where value exists
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
# Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [23]:
result=pd.read_csv('DataFrame.csv')
result.Survival = result.Survival.astype(int)
result

,Subject,Path,Survival
0,Alessandrini,ALESSANDRINI_GLAUCO,0
1,Angeloni,ANGELONI_GIUSEPPINA,1
2,Assanto,ASSANTO_MARIA,0
3,Asta,ASTA_G,1
4,Bagnoli,BAGNOLI_VINCENZO,1
...,...,...,...
148,Vitulano,VITULANO_RITA,1
149,Zampella,ZAMPELLA_A,1
150,Zanatta,ZANATTA_CARLO,1
151,Zangari,ZANGARI_ALDO,0


# Matrix construction

In [9]:
#Data loading
listOfElems=[]  #array that need to obtain the effective subjects with the sequence that we want to analyse 
Dim = []  #array that need to contain the dimension of each image in order to check the most frequent or the minimum one
Data = [] #array that contains information of the image after the extraction of data from the nibabel format
IMG=[] #array that contains the "file.nii.gz" information thanks to the nibabel module 
for Path in result["Path"]: #here starts the iteration on all the paths written in the column "Path" in result dataframe
    #print(Path)
    if os.path.isfile(f"{datasetDir}/{Path}/{SEQUENCE_1}.nii"): #I need just the Paths that contain a certain sequence

        IMG_reg = nb.load(f"{datasetDir}/{Path}/{SEQUENCE_1}.nii") #nobabel module that allow the loading of nifty file
        DATA= IMG_reg.get_data() #get_data takes the information about scale of gray
        a = [DATA.shape]
        Dim.append(a) #allows the array construction
        Data.append(DATA)  #allows the array construction
        IMG.append(IMG_reg)  #allows the array construction
        

        IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/{MaskPath}.nii") #that's the file of the mask that limits information only on a certain region of the tumor
        ROI_DATA=IMG_roi.get_data()
        b=[ROI_DATA.shape]
        Dim.append(b)
        Data.append(ROI_DATA)
        IMG.append(IMG_roi)
        
        path=[f"{Path}"]
        listOfElems.append(path) #allows the array construction
        
    else: #anyway there are some patient with the sequence with different name but with the same information, so I can include them with the "else"
        if os.path.isfile(f"{datasetDir}/{Path}/{SEQUENCE_2}.nii"):
            IMG_reg = nb.load(f"{datasetDir}/{Path}/{SEQUENCE_2}.nii")
            DATA= IMG_reg.get_data()
            a = [DATA.shape]
            Dim.append(a)
            Data.append(DATA)
            IMG.append(IMG_reg)
            
            IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/{MaskPath}.nii")
            ROI_DATA=IMG_roi.get_data()
            b=[ROI_DATA.shape]
            Dim.append(b)
            Data.append(ROI_DATA)
            IMG.append(IMG_roi)
            
            path=[f"{Path}"]
            listOfElems.append(path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_dat

In [10]:
len(listOfElems)

102

In [24]:
'''Labels Array Construction'''

#the following iteration allows the extraction of indexes corresponding to the paths selected by the preavious iterations
T1_Subject=[]
for i in range(len(listOfElems)):
    Pos=getIndexes(result, listOfElems[i][0])
    print(Pos[0][0])
    T1_Subject.append(Pos[0][0])  #the T1_Subject array contains these information

#the following iteration takes the corresponding Survival information on the result Dataframe built before    
T1_Subject_array=np.asarray(T1_Subject)

T1_Labels=[]
for i in range(len(T1_Subject_array)):
    lab = result.loc[T1_Subject_array[i],"Survival"]
    T1_Labels.append(int(lab)) #creates the list that contains these information

T1_Labels=np.asarray(T1_Labels) #that's the relative array
#T1_Labels

#the following iteration allows the construction of the final labels array with doubled information of each element of T1_Labels array
Label_Def=[]
for x in range(len(T1_Labels)):
    label_Def=[[T1_Labels[x]]*2]
    Label_Def.append(label_Def)
    
Label_Def=np.asarray(Label_Def)
Label_Def=np.ravel(Label_Def)
Label_Def #that's the final label array that can be used for the training

0
2
7
9
10
11
13
16
17
18
19
20
21
22
24
26
28
30
36
37
39
40
41
43
44
45
46
48
49
50
51
52
56
57
58
59
60
61
63
64
65
66
67
69
70
71
72
73
74
75
76
77
78
79
80
82
85
86
87
88
91
92
94
96
98
99
100
101
102
104
107
110
113
114
115
116
117
118
120
121
122
123
124
125
126
127
128
131
134
135
136
137
138
139
140
142
143
145
146
148
150
152


array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0])

In [12]:
'''Working on dimensions'''

#in order to work on dimensions it's necessary the array construction (as predicted in previous blocks)
Dim=np.asarray(Dim)   
Data=np.asarray(Data)
IMG=np.asarray(IMG)

#in order to know the minimum dimensions
Min_value=np.amin(Dim, axis=1)
Min_value
Min=np.amin(Min_value, axis=0)
Min_value.shape[0]

not_in_index = [k for k in range(Min_value.shape[0]) if not np.all(Min_value[k] == (dim1, dim2, dim3))] #that's the
#construction of an array that contains the position not corresponding to the dimension researched [(192,256,144)
#in this case]

pos_1=np.where(Min_value[:,0]==dim1) #position with first dimension equal to 192
pos_2=np.where(Min_value[:,1]==dim2) #position with first dimension equal to 256
pos_3=np.where(Min_value[:,2]==dim3) #position with first dimension equal to 144
eq=np.intersect1d(pos_1,pos_2)  #that command in order to find the intersection between the pos_1 and pos_2
index_IMG=np.intersect1d(eq,pos_3) #intersection that gives the complememntary information of not_in_index

#Here is given a random position that corresponds to the dimension request
def_index=random.choice(index_IMG)
print(def_index)


91


In [13]:
'''RESAMPLE BLOCK'''

#The resample function is executed only on images without the dimension request, respect to a random image with dimension (192,256,144)
for i in not_in_index:
    Res=nilearn.image.resample_to_img(IMG[i], IMG[def_index])
    IMG[i]=Res
    Data[i]=IMG[i].get_data()

/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyar

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nilearn/image/resampling.py:513: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/.local/lib/python3.6/site-packages/nile

In [14]:
'''Reshaping of Input Matrix'''

Input_matrix=np.empty((len(Data),dim1,dim2,dim3)) #in order to generate an empty array with a fixed shape

for i in not_in_index:

    Input_matrix[i,:,:,:]=np.array(Data[i])




for i in index_IMG:

    Input_matrix[i,:,:,:]=np.array(Data[i])
    
    
#in order to check the correct construction    
Input_matrix.shape

#import pickle
#pickle.dump( Input_matrix, open( "Input_matrix.pickle", "wb" ) )

(204, 192, 256, 144)

# TRAINING

In [15]:
'''Modules needed'''

import tensorflow as tf
from keras.models import Sequential

from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.optimizers import Adam, SGD

from keras import backend as K

Using TensorFlow backend.


In [25]:
## Create list of indices and shuffle them
N = Input_matrix.shape[0]
indices = np.arange(N)
np.random.shuffle(indices)

#  Cut the dataset at 80% to create the training and test set
N_80p = int(p * N)
indices_train = indices[:N_80p]
indices_test = indices[N_80p:]

# Split the data into training and test sets
X_train = Input_matrix[indices_train, ...]
X_test = Input_matrix[indices_test, ...]

print(X_train.shape, X_test.shape)

labels=Label_Def

#Outcome variable block added
y_train = labels[indices_train] == 0
y_test = labels[indices_test] == 1

from keras.utils import to_categorical #Convert a class vector (integer) into binary class matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


X_train.shape

(163, 192, 256, 144) (41, 192, 256, 144)


(163, 192, 256, 144)

In [22]:
y_train.shape

(163, 2)

In [17]:
'''Parameters setting'''

# Get shape of input data
data_shape = tuple(X_train.shape[1:])

data_shape

(192, 256, 144)

# Model

In [18]:
#model block added
K.clear_session()
model = Sequential()

model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=data_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 2, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 4, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

# optimizer
learning_rate = 1e-5
adam = Adam(lr=learning_rate)
sgd = SGD(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=adam, # swap out for sgd 
              metrics=['accuracy'])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 190, 254, 16)      20752     
_________________________________________________________________
batch_normalization_1 (Batch (None, 190, 254, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 95, 127, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 93, 125, 32)       4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 93, 125, 32)       128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 46, 62, 32)        0         
______________________________________

In [28]:
'''Model fitting'''

%time fit = model.fit(X_train, y_train, epochs=nEpochs, batch_size=batch_size)

Epoch 1/100


InternalError: 2 root error(s) found.
  (0) Internal: Dst tensor is not initialized.
	 [[{{node _arg_conv2d_1_input_0_0}}]]
	 [[metrics/accuracy/Identity/_217]]
  (1) Internal: Dst tensor is not initialized.
	 [[{{node _arg_conv2d_1_input_0_0}}]]
0 successful operations.
0 derived errors ignored.

# Model plot

In [ ]:
fig = plt.figure(figsize=(10, 4))
epoch = np.arange(nEpochs) + 1
fontsize = 16
plt.plot(epoch, fit.history['accuracy'], marker="o", linewidth=2,
         color="steelblue", label="accuracy")
plt.plot(epoch, fit.history['loss'], marker="o", linewidth=2,
         color="orange", label="loss")
plt.xlabel('epoch', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(frameon=False, fontsize=16);